In [1]:
# Implementation of https://www.jstor.org/stable/25734098

import pandas as pd
import numpy as np
import pymc as pm

C:\Users\theal\AppData\Local\Temp\ipykernel_12116\965775754.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
df = pd.read_csv('./data/train_with_dummies.csv', index_col=[0])
df.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,1,0,0,0,0,1,0


In [4]:
df.shape

(1460, 290)

In [ ]:
# Example data: y (observations), X (design matrix)
np.random.seed(42)
n_samples, n_features = 100, 10
X = np.random.randn(n_samples, n_features)
true_beta = np.zeros(n_features)
true_beta[0:2] = np.array([1, -1])  # Only the first two features are non-zero
y = X @ true_beta + np.random.randn(n_samples) * 0.1

with pm.Model() as model:
    # Priors
    lambda_ = pm.HalfCauchy('lambda_', beta=1, shape=n_features)
    tau = pm.HalfCauchy('tau', beta=1)
    beta = pm.Normal('beta', mu=0, sigma=tau * lambda_, shape=n_features)
    
    # Likelihood
    y_obs = pm.Normal('y_obs', mu=pm.math.dot(X, beta), sigma=0.1, observed=y)
    
    # Inference
    trace = pm.sample(1000, return_inferencedata=True)